In [23]:
import numpy as np
import pandas as pd
import pickle
import datetime
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import re

In [5]:
node_list = np.load('./node_list.npy')
geo_meta_info = pd.read_csv('./geo_meta_info.csv', sep='\t')
meta_info = pd.read_csv('./meta_info.csv', sep='\t')
df_incident = pd.read_csv('./incidents_processed.csv', sep='\t')

C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3308: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
meta_info

,Fwy,District,County,City,CA PM,Abs PM,Length,ID,Name,Lanes,Type,Sensor Type,HOV,MS ID,IRM
0,SR1-N,12,Orange,NaN,12.39,12.401,2.695,1221176,REEF POINT DR,1,Mainline,NaN,No,lue,NaN
1,SR1-N,12,Orange,NaN,12.78,12.791,0.485,1221170,CRYSTAL HEIGHTS,1,Mainline,NaN,No,2553,NaN
2,SR1-N,12,Orange,NaN,13.36,13.371,1.825,1221163,1 NB LOS TRANCOS,1,Mainline,NaN,No,2552,NaN
3,SR1-N,12,Orange,Newport Beach,16.43,16.441,1.825,1221148,1 NB AVOCADO AVE,1,Mainline,NaN,No,2551,NaN
4,SR1-N,12,Orange,Newport Beach,17.01,17.021,0.595,1221143,1 NB IRVINE TERRACE,1,Mainline,NaN,No,2550,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18698,I980-W,4,Alameda,Oakland,.98,0.971,NaN,408594,18th St off,3,Off Ramp,loops,No,DT908,z3-W-33-980-00098
18699,I980-W,4,Alameda,Oakland,.98,0.971,0.305,401610,14th/18th St,4,Mainline,loops,No,DT908,L4-W-33-980-00098
18700,I980-W,4,Alameda,Oakland,1.31,1.301,0.320,401416,23rd St,5,Mainline,loops,No,DT413,L5-W-33-980-00131
18701,I980-W,4,Alameda,Oakland,1.62,1.611,NaN,401912,29th street,3,Fwy-Fwy,loops,No,DT415,C3-W-33-980-00162


In [7]:
df_incident

,Incident Id,Start Time,Duration (mins),Freeway,CA PM,Abs PM,Source,AREA,LOCATION,DESCRIPTION,type,nearest_node
0,21379205,01/03/2023 13:23,61,SR51-N,4.8,4.8,CHP,North Sac,El Camino Ave E Onr / Sr51 N,20002-Hit and Run No Injuries,accident,312807
1,21379238,01/03/2023 13:52,4,I580-E,9.816,26.0,CHP,Dublin,I580 E / Vasco Rd Ofr,1182-Trfc Collision-No Inj,accident,418342
2,21379243,01/03/2023 13:21,379,I80-E,0.454,13.8,CHP,Oakland,I80 E / Central Ave Onr,1183-Trfc Collision-Unkn Inj,accident,407260
3,21379245,01/03/2023 13:12,104,I80-W,4.79,10.1,CHP,Oakland,I80 W / I80 W SR13 S CON,20002-Hit and Run No Injuries,accident,400691
4,21379247,01/03/2023 12:38,60,US101-S,1.169,429.5,CHP,San Francisco FSP,Us101 S / Paul Ave Ofr,1182-Trfc Collision-No Inj,accident,403092
...,...,...,...,...,...,...,...,...,...,...,...,...
477571,22105233,12/31/23 22:56,0,I8-W,R4.757,82.1,CHP,El Centro,0 I8 W,1013-Road/Weather Conditions,weather,1113364
477572,22105662,01/01/2024 6:26,703,I15-S,44.522,152.4,CHP,Victorville,I15 S / Stoddard Wells Rd,1013-Road/Weather Conditions,weather,820894
477573,22105705,01/01/2024 7:32,7,SR17-S,0.047,12.6,CHP,Santa Cruz,Sr17 S / Summit,1013-Road/Weather Conditions,weather,405586
477574,22105828,01/01/2024 10:05,0,I8-W,R4.757,82.1,CHP,El Centro,0 I8 W,1013-Road/Weather Conditions,weather,1113364


In [8]:
geo_meta_info

,Unnamed: 0,Road Width,Lane Width,Inner Shoulder Width,Inner Shoulder Treated Width,Outer Shoulder Width,Outer Shoulder Treated Width,Design Speed Limit,Functional Class,Inner Median Type,Inner Median Width,Terrain,Population,Barrier,Surface,Roadway Use,Lat,Lng,station_id
0,0,56 ft,11.2 ft,2 ft,2 ft,10 ft,10 ft,70 mph,Principal Arterial W/ C/L Prin Arterial,Paved - No Roadway Use,6 ft,Flat,Urbanized,Concrete Barrier,Bridge Deck,Median Lane is HOV Lane,33.992148,-118.069414,762624
1,1,48 ft,12.0 ft,8 ft,8 ft,10 ft,10 ft,70 mph,Principal Arterial W/ C/L Prin Arterial,Separate Grades,99 ft,Mountainous,Rural,No Barriers,Concrete,No Special Features,37.740132,-121.588291,426775
2,2,55 ft,11.0 ft,3 ft,3 ft,10 ft,10 ft,70 mph,Principal Arterial W/ C/L Prin Arterial,Paved - No Roadway Use,8 ft,Flat,Urbanized,Concrete Barrier w/Glare Screen,Bridge Deck,No Special Features,34.156500,-118.422723,769359
3,3,12 ft,12.0 ft,1 ft,1 ft,8 ft,8 ft,65 mph,Principal Arterial W/ C/L Prin Arterial,Paved - No Roadway Use,2 ft,Flat,Rural,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,38.140802,-122.221723,402050
4,4,48 ft,12.0 ft,5 ft,5 ft,10 ft,10 ft,70 mph,Principal Arterial W/ C/L Prin Arterial,Paved - No Roadway Use,36 ft,Flat,Urbanized,Concrete Barrier,Bridge Deck,NaN,34.066490,-117.246445,816401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18698,19500,24 ft,12.0 ft,5 ft,5 ft,8 ft,8 ft,70 mph,Principal Arterial W/ C/L Prin Arterial,Paved - No Roadway Use,22 ft,Flat,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,36.326780,-119.207732,602524
18699,19501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.754868,-119.750505,603108
18700,19502,48 ft,12.0 ft,8 ft,8 ft,10 ft,10 ft,70 mph,Principal Arterial W/ C/L Prin Arterial,Unpaved,36 ft,Rolling,Rural,Metal Beam Barrier,Concrete,No Special Features,38.310730,-122.033429,411137
18701,19503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.134842,-117.339254,814547


In [20]:
road_surface = {geo_meta_info['station_id'].iloc[i]: geo_meta_info['Surface'].iloc[i] for i in range(len(geo_meta_info))}
road_terrain = {geo_meta_info['station_id'].iloc[i]: geo_meta_info['Terrain'].iloc[i] for i in range(len(geo_meta_info))}
road_width = {geo_meta_info['station_id'].iloc[i]: geo_meta_info['Road Width'].iloc[i] for i in range(len(geo_meta_info))}
Fwy2id, Fwy2district = {}, {}
for i in range(len(meta_info)):
    Fwy2id[meta_info['Fwy'].iloc[i]] = meta_info['ID'].iloc[i]
    Fwy2district[meta_info['Fwy'].iloc[i]] = meta_info['District'].iloc[i]
print(meta_info['Type'].value_counts())
print(geo_meta_info['Surface'].value_counts())
print(geo_meta_info['Terrain'].value_counts())

Mainline                9534
On Ramp                 3706
Off Ramp                2650
HOV                     2139
Fwy-Fwy                  524
Conventional Highway     101
Coll/Dist                 49
Name: Type, dtype: int64
Concrete                      10102
Base & Surface >= 7" Thick     5466
Bridge Deck                    2011
Base & Surface < 7" Thick        72
Undetermined                      1
Oiled Earth - Gravel              1
Name: Surface, dtype: int64
Flat           12450
Rolling         4732
Mountainous      595
Name: Terrain, dtype: int64


## Find the district: 8, 3, 4, 7

In [10]:
# node_id = [716479, 801230, 1201931, 1201949]
df_selected = df_incident.copy() #df_incident[(df_incident['nearest_node'] == 801230)]
df_selected = df_selected.dropna()
df_selected['Start Time'] = pd.to_datetime(df_selected['Start Time'])
top_8_accidents = df_selected['DESCRIPTION'].value_counts().nlargest(8).index
print(top_8_accidents)

df_selected['date'] = df_selected['Start Time'].dt.date
df_selected['hour'] = df_selected['Start Time'].dt.hour
days = pd.date_range(start='2023-01-01', end='2023-12-31')
hours = range(24)
map_day = {datetime.date.strftime(days[i], '%Y-%m-%d'):i for i in range(len(days))}
df_selected['District'] = [0] * len(df_selected)

for i in range(len(df_selected)):
    if df_selected['DESCRIPTION'].iloc[i] in top_8_accidents and datetime.datetime.strftime(df_selected['date'].iloc[i], '%Y-%m-%d') in map_day:
        if df_selected['Freeway'].iloc[i] in Fwy2id:
            Fwyid = Fwy2id[df_selected['Freeway'].iloc[i]]
            df_selected['District'].loc[i] = Fwy2district[df_selected['Freeway'].iloc[i]]

Index(['1125-Traffic Hazard', '1183-Trfc Collision-Unkn Inj',
       '1182-Trfc Collision-No Inj', '1179-Trfc Collision-1141 Enrt',
       '20002-Hit and Run No Injuries', 'FIRE-Report of Fire',
       '1125A-Animal Hazard', 'CZP-Assist with Construction'],
      dtype='object')


C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [11]:
df_selected

,Incident Id,Start Time,Duration (mins),Freeway,CA PM,Abs PM,Source,AREA,LOCATION,DESCRIPTION,type,nearest_node,date,hour,District
0,21379205,2023-01-03 13:23:00,61,SR51-N,4.8,4.8,CHP,North Sac,El Camino Ave E Onr / Sr51 N,20002-Hit and Run No Injuries,accident,312807,2023-01-03,13,3
1,21379238,2023-01-03 13:52:00,4,I580-E,9.816,26.0,CHP,Dublin,I580 E / Vasco Rd Ofr,1182-Trfc Collision-No Inj,accident,418342,2023-01-03,13,4
2,21379243,2023-01-03 13:21:00,379,I80-E,0.454,13.8,CHP,Oakland,I80 E / Central Ave Onr,1183-Trfc Collision-Unkn Inj,accident,407260,2023-01-03,13,3
3,21379245,2023-01-03 13:12:00,104,I80-W,4.79,10.1,CHP,Oakland,I80 W / I80 W SR13 S CON,20002-Hit and Run No Injuries,accident,400691,2023-01-03,13,3
4,21379247,2023-01-03 12:38:00,60,US101-S,1.169,429.5,CHP,San Francisco FSP,Us101 S / Paul Ave Ofr,1182-Trfc Collision-No Inj,accident,403092,2023-01-03,12,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477571,22105233,2023-12-31 22:56:00,0,I8-W,R4.757,82.1,CHP,El Centro,0 I8 W,1013-Road/Weather Conditions,weather,1113364,2023-12-31,22,0
477572,22105662,2024-01-01 06:26:00,703,I15-S,44.522,152.4,CHP,Victorville,I15 S / Stoddard Wells Rd,1013-Road/Weather Conditions,weather,820894,2024-01-01,6,0
477573,22105705,2024-01-01 07:32:00,7,SR17-S,0.047,12.6,CHP,Santa Cruz,Sr17 S / Summit,1013-Road/Weather Conditions,weather,405586,2024-01-01,7,0
477574,22105828,2024-01-01 10:05:00,0,I8-W,R4.757,82.1,CHP,El Centro,0 I8 W,1013-Road/Weather Conditions,weather,1113364,2024-01-01,10,0


In [12]:
df_incident

,Incident Id,Start Time,Duration (mins),Freeway,CA PM,Abs PM,Source,AREA,LOCATION,DESCRIPTION,type,nearest_node
0,21379205,01/03/2023 13:23,61,SR51-N,4.8,4.8,CHP,North Sac,El Camino Ave E Onr / Sr51 N,20002-Hit and Run No Injuries,accident,312807
1,21379238,01/03/2023 13:52,4,I580-E,9.816,26.0,CHP,Dublin,I580 E / Vasco Rd Ofr,1182-Trfc Collision-No Inj,accident,418342
2,21379243,01/03/2023 13:21,379,I80-E,0.454,13.8,CHP,Oakland,I80 E / Central Ave Onr,1183-Trfc Collision-Unkn Inj,accident,407260
3,21379245,01/03/2023 13:12,104,I80-W,4.79,10.1,CHP,Oakland,I80 W / I80 W SR13 S CON,20002-Hit and Run No Injuries,accident,400691
4,21379247,01/03/2023 12:38,60,US101-S,1.169,429.5,CHP,San Francisco FSP,Us101 S / Paul Ave Ofr,1182-Trfc Collision-No Inj,accident,403092
...,...,...,...,...,...,...,...,...,...,...,...,...
477571,22105233,12/31/23 22:56,0,I8-W,R4.757,82.1,CHP,El Centro,0 I8 W,1013-Road/Weather Conditions,weather,1113364
477572,22105662,01/01/2024 6:26,703,I15-S,44.522,152.4,CHP,Victorville,I15 S / Stoddard Wells Rd,1013-Road/Weather Conditions,weather,820894
477573,22105705,01/01/2024 7:32,7,SR17-S,0.047,12.6,CHP,Santa Cruz,Sr17 S / Summit,1013-Road/Weather Conditions,weather,405586
477574,22105828,01/01/2024 10:05,0,I8-W,R4.757,82.1,CHP,El Centro,0 I8 W,1013-Road/Weather Conditions,weather,1113364


In [13]:
mapping = df_incident.groupby('nearest_node')['Freeway'].unique().reset_index()

In [14]:
df_selected['District'].value_counts()

8     109113
3     106662
4      95669
7      73070
0      45679
11     18464
12      8813
10      7271
6       6078
5         80
Name: District, dtype: int64

## Data Preprocessing: Create Node_data

In [15]:
df_district = df_selected[(df_selected['District'] == 8) | (df_selected['District'] == 3) | (df_selected['District'] == 4) | (df_selected['District'] == 7)]
df_district = df_district[df_district['nearest_node'] != -1]
df_district = df_district.groupby(['nearest_node', 'District']).filter(lambda x: len(x) > 1000)
top_8_accidents = df_district['DESCRIPTION'].value_counts().nlargest(8).index
selected_nodes = df_district['nearest_node'].unique()

In [16]:
node2col = {node_list[i]:i for i in range(len(node_list))}
node_data = {node:np.zeros((0, 3)) for node in node_list}
for i in range(1, 13):
    file_path = f'./m_traffic/p{str(i).zfill(2)}_done.npy'
    print(i, file_path)
    month_data = np.load(file_path, mmap_mode='r')
    for node in selected_nodes:
        node_col = node2col[node]
        node_data[node] = np.concatenate((node_data[node], month_data[:, node_col, :]), axis=0)

for node in selected_nodes:
    np.save(f'./node_data/{node}_node_data.npy', node_data[node])

1 ./entire_data_v3/p01_done.npy
2 ./entire_data_v3/p02_done.npy


KeyboardInterrupt: 

## Data preprocessing: Create DAG data

In [69]:
days = pd.date_range(start='2023-01-01', end='2023-12-31')
hours = range(24)
map_day = {datetime.date.strftime(days[i], '%Y-%m-%d'):i for i in range(len(days))}

# --------------------------------- 0-7 Accident --
#------------------------------
district_node2id = {8:{}, 3:{}, 4:{}, 7:{}}
district_id2node = {8:{}, 3:{}, 4:{}, 7:{}}
node2fwy = {}
for district in [8, 3, 4, 7]:
    node_size = 0
    for node in selected_nodes:
        if np.any((df_district['nearest_node'] == node) & (df_district['District'] == district)):
            district_node2id[district][node] = len(district_node2id[district])
            district_id2node[district][len(district_node2id[district]) - 1] = node
district_array = {}
for district in [8, 3, 4, 7]:
    district_array[district] = np.zeros((len(days) * len(district_node2id[district]), 24, 18))
map_accident = {top_8_accidents[i]:i for i in range(len(top_8_accidents))}
for i in range(len(df_district)):
    if df_district['DESCRIPTION'].iloc[i] in top_8_accidents and datetime.datetime.strftime(df_district['date'].iloc[i], '%Y-%m-%d') in map_day:
        if df_selected['Freeway'].iloc[i] in Fwy2id:
            duration = int(df_district['Duration (mins)'].iloc[i])
            hour = df_district['hour'].iloc[i]
            node = df_district['nearest_node'].iloc[i]
            day = map_day[datetime.datetime.strftime(df_district['date'].iloc[i], '%Y-%m-%d')]
            accident = map_accident[df_district['DESCRIPTION'].iloc[i]]
            district = df_district['District'].iloc[i]
            fwy = df_district['Freeway'].iloc[i]
            baserow = len(days) * district_node2id[district][node]
            node2fwy[node] = fwy
            while duration > 0 and hour < 24:
                district_array[district][baserow + day, hour, accident] = min(1, duration / 60)
                hour += 1
                duration -= 60

# --------------------------------- 8:Weekday, 9:Event, 10:Location, 11:Surface 12:Terrain 13:Width ---------------------------------

event_days = [
    '2023-01-01',
    '2023-01-02',
    '2023-01-16',
    '2023-02-20',
    '2023-04-09',
    '2023-05-29',
    '2023-07-04',
    '2023-09-04',
    '2023-10-09',
    '2023-11-11',
    '2023-11-23',
    '2023-12-25'
]

Surface2id, Terrain2id = {}, {}
for district in [8, 3, 4, 7]:
    for i in range(len(district_node2id[district])):
        node = district_id2node[district][i]
        for day in range(365):
            for hour in range(24):
                district_array[district][i * len(days) + day, hour, 8] = days[day].dayofweek >= 5
                district_array[district][i * len(days) + day, hour, 9] = days[day].strftime('%Y-%m-%d') in event_days
            if road_surface[Fwy2id[node2fwy[node]]] not in Surface2id:
                Surface2id[road_surface[Fwy2id[node2fwy[node]]]] = len(Surface2id)
            district_array[district][i * len(days) + day, Surface2id[road_surface[Fwy2id[node2fwy[node]]]], 11] = 1
            if road_terrain[Fwy2id[node2fwy[node]]] not in Terrain2id:
                Terrain2id[road_terrain[Fwy2id[node2fwy[node]]]] = len(Terrain2id)
            district_array[district][i * len(days) + day, Terrain2id[road_terrain[Fwy2id[node2fwy[node]]]], 12] = 1
            if isinstance(road_width[Fwy2id[node2fwy[node]]], float) and np.isnan(road_width[Fwy2id[node2fwy[node]]]):
                district_array[district][i * len(days) + day, 0, 13] = 0
            else:
                district_array[district][i * len(days) + day, 0, 13] = int(re.findall(r'\d+', road_width[Fwy2id[node2fwy[node]]])[0])


for i in range(len(df_district)):
    if df_district['DESCRIPTION'].iloc[i] in top_8_accidents and datetime.datetime.strftime(df_district['date'].iloc[i], '%Y-%m-%d') in map_day:
        if df_selected['Freeway'].iloc[i] in Fwy2id:
            duration = int(df_district['Duration (mins)'].iloc[i])
            hour = df_district['hour'].iloc[i]
            day = map_day[datetime.datetime.strftime(df_district['date'].iloc[i], '%Y-%m-%d')]
            district = df_district['District'].iloc[i]
            is_con = 'con' in df_district['LOCATION'].iloc[i].lower()
            Fwyid = Fwy2id[df_selected['Freeway'].iloc[i]]
            node = df_district['nearest_node'].iloc[i]
            baserow = len(days) * district_node2id[district][node]
            while duration > 0 and hour < 24:
                if is_con is True:
                    district_array[district][baserow + day, hour, 10] = min(1, duration / 60)
                else:
                    district_array[district][baserow + day, hour, 10] = - min(1, duration / 60)
                hour += 1
                duration -= 60

# --------------------------------- 13: Flow, 14: Occupancy, 15: Speed ---------------------------------

for district in [8, 3, 4, 7]:
    node_size = 0
    for node in selected_nodes:
        if np.any((df_district['nearest_node'] == node) & (df_district['District'] == district)):
            print(district, node)
            node_data = np.load(f'./node_data/{node}_node_data.npy')
            node_data = np.nan_to_num(node_data)
            node_size += 1
            baserow = district_node2id[district][node] * len(days)
            for day in range(365):
                for hour in range(24):
                    district_array[district][baserow + day, hour, 15] += np.mean(node_data[(day * 24 + hour) * 12:(day * 24 + hour + 1) * 12, 0])
                    district_array[district][baserow + day, hour, 16] += np.mean(node_data[(day * 24 + hour) * 12:(day * 24 + hour + 1) * 12, 1])
                    district_array[district][baserow + day, hour, 17] += np.mean(node_data[(day * 24 + hour) * 12:(day * 24 + hour + 1) * 12, 2])

8 1201931
8 769374
8 826060
8 1201949
8 759280
8 809294
8 769373
8 775774
8 801230
8 716479
8 1213786
3 1201931
3 769374
3 826060
3 1201949
3 759280
3 809294
3 775774
3 801230
3 716479
4 1201931
4 769374
4 826060
4 1201949
4 775774
4 801230
4 716479
7 1201931
7 1201949
7 801230


## Add Weather data

In [97]:
weather_data = pd.read_csv('f_weather.csv')
weather_data = weather_data[(weather_data['district_id'] == 'd4') | (weather_data['district_id'] == 'd7')]
weather_data = pd.concat([weather_data, pd.read_csv('p38_weather.csv')])
for district in [8, 3, 4, 7]:
    district_array[district][:, :, 10] = 16

for i in range(len(weather_data)):
    if weather_data['district_id'].iloc[i] == 'd4':
        district = 4
    elif weather_data['district_id'].iloc[i] == 'd7':
        district = 7
    elif weather_data['district_id'].iloc[i] == 'd3':
        district = 3
    elif weather_data['district_id'].iloc[i] == 'd8':
        district = 8

    rain_description = weather_data['Weather'].iloc[i]
    dt = weather_data['dt'].iloc[i]
    day = map_day[dt[:10]]
    hour = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S').hour
    vis = int(weather_data['Visibility'].iloc[i])
    if vis == -1:
        vis = 16
    if 'Heavy rain' in rain_description:
        rain = 3
    elif 'Rain' in rain_description:
        rain = 2
    elif 'Light rain' in rain_description:
        rain = 1
    else:
        rain = 0
    for j in range(len(district_node2id[district])):
        district_array[district][j * 365 + day, hour, 14] = rain
        district_array[district][j * 365 + day, hour, 10] = vis

array([ True,  True, False, ..., False,  True,  True])

## Save district_DAG data

In [98]:
for district in [8, 3, 4, 7]:
    np.save(f'./district_{district}_DAGdata.npy', district_array[district])

In [100]:
district_array[3][:, :, 16]

array([[0.00891667, 0.01505   , 0.01445   , ..., 0.04358333, 0.039725  ,
        0.041675  ],
       [0.04348333, 0.04574167, 0.03535833, ..., 0.03940833, 0.01825833,
        0.03578333],
       [0.01685   , 0.05971667, 0.01259167, ..., 0.02966667, 0.03490833,
        0.03668333],
       ...,
       [0.04521667, 0.02618333, 0.02764167, ..., 0.052775  , 0.03705833,
        0.0589    ],
       [0.070375  , 0.06853333, 0.08481667, ..., 0.03775   , 0.02636667,
        0.05385   ],
       [0.01570833, 0.01319167, 0.050975  , ..., 0.03704167, 0.05559167,
        0.04296667]])

In [25]:
top_8_accidents

Index(['1125-Traffic Hazard', '1182-Trfc Collision-No Inj',
       '1183-Trfc Collision-Unkn Inj', '1179-Trfc Collision-1141 Enrt',
       '20002-Hit and Run No Injuries', 'FIRE-Report of Fire',
       '1125A-Animal Hazard', 'CFIRE-Car Fire'],
      dtype='object')

In [24]:
df_district[['nearest_node', 'District']].value_counts()

nearest_node  District
1201949       7           9198
1201931       7           8634
801230        8           8127
716479        7           5776
775774        7           5468
826060        8           5401
769374        4           5290
809294        8           5074
759280        8           4780
1213786       7           4231
769373        4           4043
776229        7           3481
773393        7           2713
716987        3           2420
773391        8           2370
717243        8           2037
825735        8           2035
825733        8           2005
3410081       3           1903
760484        7           1878
3410084       3           1843
717000        3           1781
716238        8           1748
716001        3           1721
715991        3           1680
715920        3           1565
716237        8           1560
501010012     4           1528
716000        3           1328
716272        8           1223
402779        4           1207
1213790       7 

In [51]:
df_district.groupby(['nearest_node', 'District']).filter(lambda x: len(x) > 3500)

,Incident Id,Start Time,Duration (mins),Freeway,CA PM,Abs PM,Source,AREA,LOCATION,DESCRIPTION,type,nearest_node,date,hour,District
5,21379269,2023-01-03 14:02:00,25,I110-N,30.57,30.5,CHP,LAFSP,Sr110 N / Orange Grove Ave,1182-Trfc Collision-No Inj,accident,716479,2023-01-03,14,7
6,21379271,2023-01-03 13:58:00,20,I110-N,30.57,30.5,CHP,Central LA,Sr110 N / Orange Grove Ave,1182-Trfc Collision-No Inj,accident,716479,2023-01-03,13,7
8,21379275,2023-01-03 13:38:00,89,I405-S,16.628,40.4,CHP,LAFSP,I405 S / Artesia Blvd Ofr,1182-Trfc Collision-No Inj,accident,1201931,2023-01-03,13,7
9,21379277,2023-01-03 13:36:00,42,I405-S,16.628,40.4,CHP,South LA,I405 S / Artesia Blvd Ofr,1182-Trfc Collision-No Inj,accident,1201931,2023-01-03,13,7
12,21379285,2023-01-03 13:03:00,74,I405-S,31.628,55.4,CHP,West LA,I405 S / Wilshire Blvd Onr,20002-Hit and Run No Injuries,accident,1201931,2023-01-03,13,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472884,22073848,2023-12-15 01:46:00,16,I210-W,R31.911,32.2,CHP,Altadena,I210 W / N Santa Anita Ave,CZP-Assist with Construction,other,826060,2023-12-15,1,8
472932,22075847,2023-12-15 23:38:00,2,I10-W,R5.455,3.3,CHP,West LA,I10 W / I405 S I10 E Con,CZP-Assist with Construction,other,801230,2023-12-15,23,8
472971,22077807,2023-12-17 00:34:00,189,I210-W,R31.911,32.2,CHP,Altadena,I210 W / N Santa Anita Ave,CZP-Assist with Construction,other,826060,2023-12-17,0,8
473475,22096736,2023-12-26 23:29:00,280,I10-E,17.755,15.6,CHP,Central LA,I10 E / Sr60 E,CZP-Assist with Construction,other,759280,2023-12-26,23,8


In [41]:
Terrain2id

{'Rolling': 0, nan: 1, 'Flat': 2}